Addition

In [ ]:
def bit_add(a, b):
    return a ^ b

Multiplication

In [16]:
def bit_mul(a, b):
    result = 0
    while b > 0:
        if b & 1:           # if the lowest bit of b is 1
            result ^= a     # add the current a to result
        a <<= 1             # multiply a by 2
        b >>= 1             # divide b by 2 (move to next bit)
    return result

In [21]:
def bit_mod(a, n):
    if n & (n - 1) == 0:  # Check if n is a power of 2
        return a & (n - 1)  # Bitwise modulo for powers of 2
    else:
        return a % n         # Standard modulo for other values

In [22]:
a = bit_mul(0x57, 0x83)
b = bit_mod(a, 0x011B)

In [24]:
bin(b)

'0b1011100'

`SubBytes` transformation

In [13]:
MOD_POLY = 0x011B  # 100011011 in binary
bin(MOD_POLY)

'0b100011011'

In [ ]:
def test():
    assert bit_add(0x57, 0x83) == 0xd4, "Addition failed: add(0x57, 0x83) should be 0xd4"

True